# Data pre-processing for EEG seizure detection

The dataset is made available as multiple SMRX files generated by Spike2 analysis software. `neo` package can be used to read the data. 
There are 6 labelled seizures in the data, timestamps and associated filenames of each seizures are stored in a `list` of `dict`s for further processing

<span style="color:orange">TODO : Generate larger dataset with random non-seizure sequences. Use SMOTE to balance the classes.</span>

In [1]:
# seizure data structure
seizures = [
    {
        'filename': 'SCN-AJ_180910_183823_1001',
        'sequence': (13539, 13580),
        'padded_sequence': (13519, 13600),
    },
    {
        'filename': 'SCN-AJ_180911_063946_1003',
        'sequence': (21040, 21094),
        'padded_sequence': (21013, 21121),
    },
    {
        'filename': 'SCN-AJ_180912_004151_1006',
        'sequence': (8284, 8325 ),
        'padded_sequence': (8264, 8345),
    },
    {
        'filename': 'SCN-AJ_180912_064236_1007',
        'sequence': (9918, 9956),
        'padded_sequence': (9900, 9975),
    },
    {
        'filename': 'SCN-AJ_180912_064236_1007',
        'sequence': (11865, 11914),
        'padded_sequence': (11840, 11940),
    },
    {
        'filename': 'SCN-AJ_180912_184406_1009',
        'sequence': (1, 24),
        'padded_sequence': (1, 50),
    },
]

In [2]:
import neo
import numpy as np
import scipy as sp
import random

def generateLearningData(seizures, signal_length, sampling_rate=3000):
    '''
    Generates the learning dataset by combining the labelled seizures and
    adjacent padding to create a balanced dataset
    '''
    
    data = np.empty((signal_length * sampling_rate, 15))
    labels = np.zeros(signal_length * sampling_rate)
    sample_start = 0
    
    random.shuffle(seizures)
    
    for seizure in seizures:
        reader = neo.io.CedIO(filename='data/' + seizure['filename'] + '.smrx')
        segment = reader.read_segment(time_slice=None, lazy=True) # lazy loading to save memory
        signals = segment.analogsignals
        
        seq_start = seizure['padded_sequence'][0]
        seq_end = seizure['padded_sequence'][1]
        
        for timestep in range(seq_start, seq_end - 1):
            
            sample_end = sample_start + sampling_rate
            
            for channel, signal in enumerate(signals):
                signal_slice = signal.load(time_slice=(timestep, timestep + 1))
                signal_slice = sp.signal.resample(signal_slice, sampling_rate)
                
                sample = sample_start
                for slice_index in range(sampling_rate):
                    data[sample, channel] = signal_slice[slice_index]
                    sample += 1
                    
            # generating labels
            if timestep >= seizure['sequence'][0] and timestep + 1 <= seizure['sequence'][1]:
                labels[sample_start:sample_end] = 1
                    
            sample_start = sample_end

    return data, labels

In [3]:
raw_data, labels = generateLearningData(seizures, 495) # 495s is the total length of seizures + padding
print(raw_data.shape)
print(labels.shape)

(1485000, 15)
(1485000,)


In [4]:
# verify the class ratio
from collections import Counter
print("Original class distribution: %s" % Counter(labels))

Original class distribution: Counter({0.0: 747000, 1.0: 738000})


In [5]:
# normalize the scales of each channel
data = raw_data
for channel in range(15):
    data[:, channel] = (data[:, channel] - np.min(data[:, channel])) / (np.max(data[:, channel]) - np.min(data[:, channel]))

In [6]:
# generate data sequence for LSTM
def generateDataSequqnce(data, labels, sequence_length, step):
    X = []
    Y = []
    for start in range(0, len(data), step):
        end = start + sequence_length
        X.append(data[start:end])
        Y.append(labels[start])
    return np.array(X), np.array(Y)

In [7]:
sequence_length, step = 1, 1
X, Y = generateDataSequqnce(data, labels, sequence_length, step)
print(X.shape)
print(Y.shape)

(1485000, 1, 15)
(1485000,)


In [8]:
# train:val:test split 80:10:10
from sklearn.model_selection import train_test_split

train_data, x_test, train_labels, y_test = train_test_split(X, Y, test_size=0.1, shuffle=False)
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.11, shuffle=False)

y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')
y_test = np.asarray(y_test).astype('float32')

print("x_train:", x_train.shape)
print("y_train", y_train.shape)
print("x_val", x_val.shape)
print("y_val", y_val.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train: (1189485, 1, 15)
y_train (1189485,)
x_val (147015, 1, 15)
y_val (147015,)
x_test (148500, 1, 15)
y_test (148500,)


## Baseline classification
Fitting the model on normalized data

<span style="color:orange">TODO : Hyperparameter optimization and trying atleast one another model</span>

In [10]:
# LSTM model definition for classification
import keras
from keras.layers import LSTM, Dropout, Dense
model = keras.Sequential()
model.add(LSTM(16, input_shape = (1, 15)))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", 
              metrics=[keras.metrics.binary_accuracy],
              optimizer="adam")

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 16)                2048      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,065
Trainable params: 2,065
Non-trainable params: 0
_________________________________________________________________


In [24]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=10, callbacks=[callback])

Epoch 1/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0597 - binary_accuracy: 0.9792 - val_loss: 1.0359 - val_binary_accuracy: 0.8576
Epoch 2/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0588 - binary_accuracy: 0.9795 - val_loss: 0.8127 - val_binary_accuracy: 0.8756
Epoch 3/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0583 - binary_accuracy: 0.9796 - val_loss: 0.7480 - val_binary_accuracy: 0.8794
Epoch 4/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0578 - binary_accuracy: 0.9801 - val_loss: 0.7463 - val_binary_accuracy: 0.8802
Epoch 5/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0567 - binary_accuracy: 0.9804 - val_loss: 0.6349 - val_binary_accuracy: 0.8930
Epoch 6/10
18586/18586 [==============================] - 22s 1ms/step - loss: 0.0563 - binary_accuracy: 0.9807 - val_loss: 0.5465 - val_binary_accuracy: 0.9021
Epoch 7/10
18586/18586 [==========

In [25]:
results = model.evaluate(x_test, y_test)
results

4641/4641 [==============================] - 3s 629us/step - loss: 10.2993 - binary_accuracy: 0.5723


[10.299306869506836, 0.5722693800926208]

The model overfits and generalizes poorly on unseen test data, although has good validation performance. Very little hyperparameter optimization was performed for the model, which will be done after fixing the issue with the Butterworth bandpass filter. 

## Pre-processing
The aim is to apply two sets of pre-processing methodologies and compare their impacts - frequency-domain methods based on signal processing, and extracting statistical features from the data to be used as features. Feature selection will be done initially to reduce the dimensionality of the data and reduce computational requirements for working with larger subsets of the data.